In [2]:
%load_ext autoreload
%autoreload 2

In [3]:
import pandas as pd
import numpy as np
import sklearn
import timescaledb_model as tsdb
from analyze import get_files_infos_df, store_file
from tqdm import tqdm 
from utils import multi_read_df_from_paths

Logs of timescaledb_model go to /tmp/bourse.log


In [33]:
# files_infos_df = get_files_infos_df()
# files_infos_df.to_pickle('files_infos.pkl')

## Files Dataframe - Filtering
1 hour for each Last day of Each month

In [34]:
files_infos_df = get_files_infos_df("./files_infos.pkl")

In [35]:
np.random.seed(42)
dates_to_take = files_infos_df.groupby([files_infos_df["year_month"]]).apply(lambda x :x.sample(n=1)).reset_index(drop=True)["date"]
file_infos_df_filtered = files_infos_df[files_infos_df["date"].isin(dates_to_take)]

In [36]:
file_infos_df_filtered["hour"] = file_infos_df_filtered.index.hour
file_infos_df_filtered = file_infos_df_filtered.groupby([file_infos_df_filtered["date"], file_infos_df_filtered["hour"], file_infos_df_filtered["market"]]).first().reset_index()

In [8]:
# dfs = multi_read_df_from_paths(list(file_infos_df_filtered["path"]))

100%|██████████| 1944/1944 [00:06<00:00, 307.15it/s]


## PEA PME - Symbol NF handling

### Check if every files from PEA PME have the same prefixes

In [140]:
dfs = multi_read_df_from_paths(list(file_infos_df_filtered[file_infos_df_filtered["market"] == "peapme"] ["path"]))

Logs of timescaledb_model go to /tmp/bourse.log


 20%|█▉        | 63/317 [00:00<00:00, 589.57it/s]

100%|██████████| 317/317 [00:00<00:00, 324.93it/s]


In [141]:
def get_prefix(df: pd.DataFrame, prefix_function = lambda x : x[0:3]) -> set:
    df["prefix"] = df["symbol"].apply(prefix_function)
    return set(df["prefix"].unique())


In [142]:
# prefixes = set()
# for i, df in enumerate(dfs):
#     current_prefixes = get_prefix(df)
#     if not current_prefixes.issubset(prefixes):
#         prefixes = prefixes.union(current_prefixes)
#         # print(current_prefixes, i)
# prefixes

In [161]:
map_prefix_to_symbol_nf = {
    "1rP": lambda x: x[3:].split("_")[0] if len(x) != 15 else np.nan,
    "1rA": lambda x: x[3:],
    "1rE": lambda x: x[4:],
    "FF1": lambda x: x.split("_")[1][0:],
}

In [168]:
def filter_df(df: pd.DataFrame) -> pd.DataFrame:
    #TODO
    return df
def add_symbol_nf_column(df: pd.DataFrame, map_prefix_to_symbol_nf: dict) -> pd.DataFrame:
    df["prefix"] = df["symbol"].apply(lambda x: x[0:3])
    df["symbol_nf"] = df.apply(
        lambda x: map_prefix_to_symbol_nf.get(x["prefix"], lambda _: x["symbol"])(x["symbol"]),
        axis=1,
    )
    return df

In [172]:
df = filter_df(dfs[0])
df = add_symbol_nf_column(df, map_prefix_to_symbol_nf)
symbol_nfs = set(df["symbol_nf"].unique())

In [170]:
df["len"] = df["symbol_nf"].apply(len)
df.sort_values("len", ascending=False)

,last,volume,symbol,name,prefix,symbol_nf,len
symbol,,,,,,,
1rPALGAUXX,0.115(c),0,1rPALGAUXX,GAUSSIN,1rP,ALGAUXX,7
1rEPALMIL,19.000,42,1rEPALMIL,1000MERCIS,1rE,ALMIL,5
1rPMLCLI,2.060(c),0,1rPMLCLI,MAISON CLIO,1rP,MLCLI,5
1rPMEDCL,10.850,4395,1rPMEDCL,MEDINCELL REG,1rP,MEDCL,5
1rPALMED,6.900(c),0,1rPALMED,MEDICREA INTERN.,1rP,ALMED,5
...,...,...,...,...,...,...,...
1rPIPH,3.740,229788,1rPIPH,INNATE PHARMA,1rP,IPH,3
1rPGV,0.787,24769,1rPGV,GENOMIC VISION,1rP,GV,2
1rPSX,4.100,1,1rPSX,CS GROUP,1rP,SX,2


## CompA - Symbol NF handling